In [47]:
#Notebook which provides best troughput achievable for a given M, N and d with existence of each edge being variable
#I'll probably make this into a proper python file soon, but it's more convenient this way for now
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np

In [48]:
N = 16
dE = 8

In [49]:
#Specify the demand matrix
# demand = np.zeros((N,N))
# for i in range(N):
#     for j in range(N):
#         if i!=j:
#             demand[i][j] = dE/(N-1)


workdir="/home/studium/Documents/Code/rdcn-throughput-master/matrices/"
demand= np.loadtxt(workdir+"data-parallelism.mat", usecols=range(N))
demand = demand * dE

In [50]:
#Initialize the model
model = gp.Model("throughput")
#TODO: Learn more about tolerances and which to use
# model.Params.OptimalityTol = 1e-9
# model.Params.FeasibilityTol = 1e-3


Set parameter OptimalityTol to value 1e-08
Set parameter FeasibilityTol to value 0.0001


In [51]:
#Add edge vars
edge_vars = {}
for i in range(N):
    for j in range(N):
        edge_vars[i, j] = model.addVar(vtype= GRB.INTEGER, name=f"edge_{i}_{j}",ub=dE)

In [52]:
#Inc edges constraint
for i in range(N):
    model.addConstr(gp.quicksum(edge_vars[i,j] for j in range(N))<=dE)

In [53]:
#Outg edges constraint
for i in range(N):
    model.addConstr(gp.quicksum(edge_vars[j,i] for j in range(N))<=dE)

In [54]:
#Add flow vars for every node combination
flow_vars = {}
for i in range(N):
    for j in range(N):
        for s in range(N):
            for d in range(N):
                flow_vars[s, d, i, j] = model.addVar(vtype= GRB.CONTINUOUS, name=f"flow_{s}_{d}_{i}_{j}",lb=0)

In [55]:
#Add throughput variable
throughput = model.addVar(vtype=GRB.CONTINUOUS,name='throughput',lb=0,ub=1)

In [56]:
#Source demand constraint
for s in range(N):
    for d in range(N):
            model.addConstr(gp.quicksum(flow_vars[s, d, s, i] for i in range(N) if i!=s )>= demand[s,d]* throughput,name =f"sdconst_{s}_{d}")

In [57]:
#Dest demand constraint
for s in range(N):
    for d in range(N):
            model.addConstr(gp.quicksum(flow_vars[s, d, i, d] for i in range(N) if i!=d )>= demand[s,d]* throughput,name =f"ddconst_{s}_{d}")

In [58]:
#Flow conservation
for s in range(N):
    for d in range(N):
        for j in (j for j in range(N) if j != s and j != d):
            model.addConstr(gp.quicksum(flow_vars[s, d, i, j] for i in range(N) if i!=j )-gp.quicksum(flow_vars[s, d, j, k] for k in range(N) if k!=j)== 0,name =f"fcconst_{s}_{d}_{j}")

In [59]:
#Capacity Constraints
for i in range(N):
    for j in range(N):
        model.addConstr(gp.quicksum(gp.quicksum(flow_vars[s,d,i,j] for d in range(N)) for s in range(N))<= edge_vars[i,j],name =f"cconst_{s}_{d}")

In [60]:
#Set the objective: maximize throughput
model.setObjective(throughput, GRB.MAXIMIZE)

In [61]:
model.optimize()
print("Throughput for the given topology and the given demand matrix is:", throughput.X)
for v in model.getVars():
        if v.x != 0:
                print(v.varName, "=", v.x)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "TUXEDO OS 2")

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4400 rows, 65793 columns and 182496 nonzeros
Model fingerprint: 0xb0512df3
Variable types: 65537 continuous, 256 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-05, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [8e+00, 8e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1856 rows and 27894 columns
Presolve time: 0.22s
Presolved: 2544 rows, 37899 columns, 105922 nonzeros
Variable types: 37659 continuous, 240 integer (0 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective 1.000000e+00, 196 iterations, 0.08 seconds (0.08 w